# Student Assignments

In [250]:
from debugging_benchmark.student_assignments.repository import (
    GCDStudentAssignmentRepository,
    NPrStudentAssignmentRepository,
)
from debugging_framework.execution.exceptions import UnexpectedResultError
from debugging_framework.input.oracle import OracleResult

In [251]:
all_programs = NPrStudentAssignmentRepository().build(
    err_def={
        UnexpectedResultError: OracleResult.FAILING,
        ZeroDivisionError: OracleResult.FAILING,
        # TimeoutError: OracleResult.FAILING,
        # RecursionError: OracleResult.FAILING
    })

In [252]:
all_programs

[StudentAssignmentBenchmarkProgram(NPr1StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr2StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr3StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr4StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr5StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr6StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr7StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr8StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr9StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr10StudentAssignmentProject())]

In [253]:
programs = [program for program in all_programs if program.get_failing_inputs() != [] and program.get_passing_inputs() != []]

In [254]:
programs

[StudentAssignmentBenchmarkProgram(NPr1StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr2StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr3StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr4StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr5StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr6StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr7StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr8StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr9StudentAssignmentProject()),
 StudentAssignmentBenchmarkProgram(NPr10StudentAssignmentProject())]

In [255]:
program = programs[0]
program

StudentAssignmentBenchmarkProgram(NPr1StudentAssignmentProject())

In [256]:
oracle = program.get_oracle()

In [273]:
# oracle("22092 22934823942228230")
oracle("96 54")

(<OracleResult.FAILING: 'FAILING'>,
 debugging_framework.execution.exceptions.UnexpectedResultError('Results do not match'))

In [258]:
patterns = [
'''
exists <?NONTERMINAL> nonterm1 in start:
  exists <?NONTERMINAL> nonterm2 in start:
    (= nonterm1 nonterm2)
''',
'''
exists <?NONTERMINAL> elem in start:
    (>= (str.to.int elem) (str.to.int <?STRING>))
''',
'''
exists <?NONTERMINAL> elem_1 in start:
  exists <?NONTERMINAL> elem_2 in start:
    (> (str.to.int elem_1) (str.to.int elem_2))
'''
]

In [259]:
param["initial_inputs"].append("9 2341231283")

In [274]:
from typing import Tuple
from isla.language import Formula
from isla.language import ISLaUnparser
from avicenna.avicenna import Avicenna

param = program.to_dict()
param["initial_inputs"].append("9 2341231283")

# Initialize Avicenna with a minimum recall configuration
avicenna = Avicenna(
    **param,
    log=True,
    top_n_relevant_features=6,
    patterns=patterns,
    min_recall=.7
)

In [275]:
diagnosis: Tuple[Formula, float, float] = avicenna.explain()
print(f"Avicenna determined the following constraints for {program}:\n")
print(ISLaUnparser(diagnosis[0]).unparse())
print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")

avicenna :: 2024-05-19 15:45:37,860 :: INFO     :: Starting Avicenna with 3 failing inputs and 1 passing inputs.
avicenna :: 2024-05-19 15:45:37,901 :: INFO     :: Starting Iteration 1
root :: 2024-05-19 15:45:38,083 :: INFO     :: Determined {num(<first>), num(<one_nine>), exists(<digit>&nbsp;->&nbsp;3), num(<second>), num(<digit>), num(<integer>)} as most relevant.
root :: 2024-05-19 15:45:38,085 :: INFO     :: Added Features: {exists(<digits>), exists(<digits>&nbsp;->&nbsp;<digit>), num(<first>), exists(<maybe_digits>&nbsp;->&nbsp;<digits>), exists(<one_nine>&nbsp;->&nbsp;9), len(<digit>), exists(<digit>&nbsp;->&nbsp;3), len(<maybe_digits>), len(<first>), num(<second>)} due to high correlation.
learner :: 2024-05-19 15:45:38,107 :: INFO     :: Keeping 2 positive examples for candidate generation.
learner :: 2024-05-19 15:45:38,147 :: INFO     :: Found 79 invariant candidates.
learner :: 2024-05-19 15:45:38,148 :: INFO     :: Evaluating Recall.
learner :: 2024-05-19 15:45:39,475 :: I

Avicenna determined the following constraints for StudentAssignmentBenchmarkProgram(NPr1StudentAssignmentProject()):

exists <second> elem_1 in start:
  exists <one_nine> elem_2 in start:
    (> (str.to.int elem_1) (str.to.int elem_2))
Avicenna calculated a precision of 71.11% and a recall of 96.27%



In [276]:
print("All Learned Formulas (that meet min criteria)", end="\n\n")
cand = avicenna.get_learned_formulas()
for can in cand:
    print(
        f"Avicenna calculated a precision of {can[1] * 100:.2f}% and a recall of {can[2] * 100:.2f}%"
    )
    print(ISLaUnparser(can[0]).unparse(), end="\n\n")
    print(len(can[0]))

All Learned Formulas (that meet min criteria)

Avicenna calculated a precision of 71.11% and a recall of 96.27%
exists <second> elem_1 in start:
  exists <one_nine> elem_2 in start:
    (> (str.to.int elem_1) (str.to.int elem_2))

3
Avicenna calculated a precision of 75.56% and a recall of 95.80%
(exists <second> elem_1 in start:
   exists <one_nine> elem_2 in start:
     (> (str.to.int elem_1) (str.to.int elem_2)) and
exists <second> elem in start:
  (>= (str.to.int elem) (str.to.int "4")))

3
Avicenna calculated a precision of 68.89% and a recall of 93.94%
(exists <second> elem_1 in start:
   exists <digit> elem_2 in start:
     (> (str.to.int elem_1) (str.to.int elem_2)) and
exists <second> elem in start:
  (>= (str.to.int elem) (str.to.int "4")))

3
Avicenna calculated a precision of 60.00% and a recall of 93.94%
(exists <second> elem in start:
   (>= (str.to.int elem) (str.to.int "2")) and
exists <second> elem_1 in start:
  exists <digit> elem_2 in start:
    (> (str.to.int elem_1

In [277]:
for inp in avicenna.all_inputs:
    print(inp, inp.oracle)

3 55 FAILING
48 390 FAILING
33 1558 FAILING
52 636 FAILING
4 43 FAILING
318 2 PASSING
98 97 FAILING
8 373 FAILING
81 81 FAILING
704 4 PASSING
547 3 PASSING
81 1226 FAILING
4 1267 FAILING
227 8 FAILING
9 2341231283 FAILING
732 1 PASSING
25 26 FAILING
4 4 FAILING
8 1 PASSING
19 79 FAILING
68 167 FAILING
88 53 FAILING
4 1714 FAILING
91 1714 FAILING
616 3 PASSING
1 62 FAILING
282 3 PASSING
4 8 FAILING
2 1 PASSING
701 28 FAILING
8 301 FAILING
4 330 FAILING
3 17586 FAILING
24 48 FAILING
5 7750 FAILING
675 3 PASSING
64 3341665 FAILING
91 41 FAILING
3 38396 FAILING
64 87 FAILING
20 2 PASSING
93 513 FAILING
48 89 FAILING
80 168 FAILING
14 64 FAILING
63 32 FAILING
620 67 FAILING
322 31 FAILING
22 83 FAILING
6 71233 FAILING
3780 3780 FAILING
602 33 FAILING
987 9 FAILING
5 5 FAILING
3 6044 FAILING
41 32 FAILING
4 11 FAILING
69 62 FAILING
9 5 PASSING
432 48 FAILING
74 14 FAILING
1 57 FAILING
73 17586 FAILING
59 8 PASSING
5 217746 FAILING
51 400396 FAILING
92 6128 FAILING
98 36 FAILING
9 9 FAILING
3

In [ ]:
from typing import Tuple
from isla.language import Formula
from isla.language import ISLaUnparser

from avicenna.avicenna import Avicenna

for program in programs:
    # Convert program to dictionary format for Avicenna initialization
    param = program.to_dict()
    
    # Initialize Avicenna with a minimum recall configuration
    avicenna = Avicenna(
        **param,
    )
    
    try:
        diagnosis: Tuple[Formula, float, float] = avicenna.explain()
        print(f"Avicenna determined the following constraints for {program}:\n")
        print(ISLaUnparser(diagnosis[0]).unparse())
        print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")
    except Exception as e:
        print(f"Error during diagnosis: {e}")

In [30]:
EvalDict = {
    "middle_1": {
        "top_n_relevant_features": 3,
    },
    "middle_2": {
        "top_n_relevant_features": 3,
    },
    "expression_1": {
        "min_recall": 0.7
    },
    "markup_1": {
        "min_recall": 0.7
    },
    "markup_2": {
        "min_recall": 0.7
    }
}
names = ["middle_2", "middle_1", "c", "markup_1"]

for name in names:
    defa = EvalDict.get(name, {})  # Use {} as the default value
    param.update(defa)
    print(param)

{'a': 1, 'top_n_relevant_features': 3}
{'a': 1, 'top_n_relevant_features': 3}
{'a': 1}
{'a': 1, 'min_recall': 0.7}


In [ ]:
param = {"a": 1}

